In [1]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_utils.py", "helper_utils.py")
import importlib
import helper_utils as hu
importlib.reload(hu)

✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\homepage\스프린트미션\실습
🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\homepage\스프린트미션\실습
🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\homepage\스프린트미션\실습


<module 'helper_utils' from 'd:\\GoogleDrive\\homepage\\스프린트미션\\실습\\helper_utils.py'>

In [2]:
# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression  # 선형/다중 회귀
from sklearn.preprocessing import PolynomialFeatures, StandardScaler  # 다항 특성, 정규화
from sklearn.model_selection import train_test_split  # 데이터 분할
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)  # 다양한 예제 데이터셋
from sklearn import datasets  # 추가 데이터셋
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree  # 결정트리
from sklearn.ensemble import RandomForestClassifier  # 랜덤포레스트
from sklearn.metrics import accuracy_score, mean_squared_error  # 평가 지표

# --- 기타 라이브러리 ---
from PIL import Image  # 이미지 처리

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn  # 신경망
import torch.optim as optim
import torch.nn.functional as F  # 활성화 함수
from torch.utils.data import Dataset, DataLoader  # PyTorch 데이터셋/로더
from torchvision.transforms import v2

# --- 기타 ---
import os
import yaml
import requests
import tarfile
import shutil
import json
import signal
import random
from datetime import datetime
from datetime import timedelta
import time
from pathlib import Path
import re
from tqdm import tqdm
import numpy as np  # 수치 연산
import matplotlib.pyplot as plt  # 시각화
import pandas as pd

# --- 디바이스 설정 ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [4]:
class EncoderRNN(nn.Module):
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size  # 수정: hidden_szie → hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

  def forward(self, input_):
    embedded = self.embedding(input_)
    output, hidden = self.gru(embedded)
    return output, hidden

In [5]:
class BahdanauAttention(nn.Module):
  def __init__(self, hidden_size):
    super().__init__()
    self.W = nn.Linear(hidden_size, hidden_size, bias=False)
    self.U = nn.Linear(hidden_size, hidden_size, bias=False)
    self.V = nn.Linear(hidden_size, 1, bias=False)

  def forward(self, decoder_hidden, encoder_outputs):
    s_proj = self.W(decoder_hidden.permute(1, 0, 2))
    h_proj = self.U(encoder_outputs)
    scored = self.V(torch.tanh(s_proj + h_proj))
    attn_weights = F.softmax(scored, dim=1)
    # batch matrix multiplcation
    context = torch.bmm(attn_weights.permute(0, 2, 1), encoder_outputs)

    return context, attn_weights.squeeze(2)

In [6]:
class BahdanauAttnDecoderRNN(nn.Module):
  def __init__(self, hidden_size, output_size):
    super().__init__()
    self.embedding = nn.Embedding(output_size, hidden_size)
    self.attention = BahdanauAttention(hidden_size)
    self.gru = nn.GRU(hidden_size * 2, hidden_size, batch_first=True)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, decoder_input, decoder_hidden, encoder_outputs):
    embedded = self.embedding(decoder_input)
    context, attn_weights = self.attention(decoder_hidden, encoder_outputs)
    rnn_input = torch.cat((embedded, context), dim=2)
    output, hidden = self.gru(rnn_input, decoder_hidden)
    output = F.log_softmax(self.out(output.squeeze(1)), dim=1)

    return output, hidden, attn_weights

In [7]:
def train_bahdanau(input_tensor, target_tensor, encoder, decoder, optimizer, criterion):
    encoder_optimizer, decoder_optimizer = optimizer
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_outputs, encoder_hidden = encoder(input_tensor)

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    loss = 0
    target_length = target_tensor.size(1)

    for di in range(target_length):
        decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
        loss += criterion(decoder_output, target_tensor[:, di])
        decoder_input = target_tensor[:, di].unsqueeze(1)

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [8]:
INPUT_FORMATS = [
    '%B %d, %Y',  # June 18, 2025
    '%d %b %Y',   # 18 Jun 2025
    '%Y/%m/%d',   # 2025/06/18
    '%m-%d-%Y'    # 06-18-2025
]

In [9]:
OUTPUT_FORMATS = '%Y-%m-%d'

In [10]:
def create_dataset(n_examples):
    """모든 가능한 월 이름을 포함한 포괄적인 데이터셋 생성"""
    dataset = []

    # 모든 월 이름과 축약형을 명시적으로 포함
    full_months = ['January', 'February', 'March', 'April', 'May', 'June',
                   'July', 'August', 'September', 'October', 'November', 'December']

    short_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    for i in range(n_examples):
        year = random.randint(1990, 2030)
        month = (i % 12) + 1
        day = random.randint(1, 28)

        current_date = datetime(year, month, day)

        # 포맷 선택을 더 균등하게
        format_choice = i % 4

        if format_choice == 0:  # 전체 월 이름
            input_str = f"{full_months[month-1]} {day:02d}, {year}"
        elif format_choice == 1:  # 축약 월 이름
            input_str = f"{day:02d} {short_months[month-1]} {year}"
        elif format_choice == 2:  # 숫자 슬래시
            input_str = f"{year}/{month:02d}/{day:02d}"
        else:  # 숫자 하이픈
            input_str = f"{month:02d}-{day:02d}-{year}"

        output_str = current_date.strftime('%Y-%m-%d')
        dataset.append((input_str, output_str))

    random.shuffle(dataset)
    return dataset

In [11]:
raw_data = create_dataset(5000)

In [12]:
raw_data[:10]

[('05 Jun 1992', '1992-06-05'),
 ('12-01-2009', '2009-12-01'),
 ('08-13-2018', '2018-08-13'),
 ('2017/11/25', '2017-11-25'),
 ('1997/07/06', '1997-07-06'),
 ('2020/03/18', '2020-03-18'),
 ('September 19, 2024', '2024-09-19'),
 ('1992/07/22', '1992-07-22'),
 ('May 10, 2013', '2013-05-10'),
 ('September 13, 1995', '1995-09-13')]

In [13]:
# 문자 단위 토큰화를 위한 새로운 Lang 클래스
class CharLang:
    def __init__(self, name):
        self.name = name
        self.char2index = {}
        self.index2char = {0: "SOS", 1: "EOS"}
        self.n_chars = 2

    def add_sentence(self, sentence):
        for char in sentence:
            if char not in self.char2index:
                self.char2index[char] = self.n_chars
                self.index2char[self.n_chars] = char
                self.n_chars += 1

# 문자 단위 언어 클래스 생성
date_input_lang = CharLang("date_input")
date_output_lang = CharLang("date_output")

# 문자 단위로 vocabulary 구축
for pair in raw_data:
    date_input_lang.add_sentence(pair[0])
    date_output_lang.add_sentence(pair[1])


In [14]:
print(f"Input character vocabulary size: {date_input_lang.n_chars}")
print(f"Output character vocabulary size: {date_output_lang.n_chars}")
print("date_input_lang", date_input_lang.char2index)
print("date_output_lang", date_output_lang.char2index)

# date_input_lang 상세 확인
print("=== date_input_lang 분석 ===")
print(f"Language name: {date_input_lang.name}")
print(f"Total characters: {date_input_lang.n_chars}")
print()

# print("Character to Index mapping:")
# for char, idx in sorted(date_input_lang.char2index.items()):
#     print(f"'{char}' → {idx}")
# print()

print("Index to Character mapping:")
for idx, char in sorted(date_input_lang.index2char.items()):
    if idx <= 10:  # 처음 몇 개만 출력
        print(f"{idx} → '{char}'")
print()

# 샘플 문장들이 어떻게 토큰화되는지 확인
print("=== 샘플 입력 문장들의 토큰화 ===")
sample_inputs = [
    "June 18, 2025",
    "18 Jun 2025",
    "2025/06/18",
    "06-18-2025"
]

for sample in sample_inputs:
    print(f"Input: '{sample}'")
    char_list = [char for char in sample]
    char_indices = [date_input_lang.char2index.get(char, -1) for char in sample]
    print(f"Character list: {char_list}")
    print(f"Character indices: {char_indices}")

    # 역변환 테스트
    reconstructed = ''.join([date_input_lang.index2char.get(idx, '?') for idx in char_indices if idx != -1])
    print(f"Reconstructed: '{reconstructed}'")
    print(f"Match: {sample == reconstructed}")
    print("-" * 50)

Input character vocabulary size: 32
Output character vocabulary size: 13
date_input_lang {'0': 2, '5': 3, ' ': 4, 'J': 5, 'u': 6, 'n': 7, '1': 8, '9': 9, '2': 10, '-': 11, '8': 12, '3': 13, '7': 14, '/': 15, '6': 16, 'S': 17, 'e': 18, 'p': 19, 't': 20, 'm': 21, 'b': 22, 'r': 23, ',': 24, '4': 25, 'M': 26, 'a': 27, 'y': 28, 'F': 29, 'O': 30, 'c': 31}
date_output_lang {'1': 2, '9': 3, '2': 4, '-': 5, '0': 6, '6': 7, '5': 8, '8': 9, '3': 10, '7': 11, '4': 12}
=== date_input_lang 분석 ===
Language name: date_input
Total characters: 32

Index to Character mapping:
0 → 'SOS'
1 → 'EOS'
2 → '0'
3 → '5'
4 → ' '
5 → 'J'
6 → 'u'
7 → 'n'
8 → '1'
9 → '9'
10 → '2'

=== 샘플 입력 문장들의 토큰화 ===
Input: 'June 18, 2025'
Character list: ['J', 'u', 'n', 'e', ' ', '1', '8', ',', ' ', '2', '0', '2', '5']
Character indices: [5, 6, 7, 18, 4, 8, 12, 24, 4, 10, 2, 10, 3]
Reconstructed: 'June 18, 2025'
Match: True
--------------------------------------------------
Input: '18 Jun 2025'
Character list: ['1', '8', ' ', 'J'

In [15]:
def create_weighted_criterion(output_lang, device=device):
    # 클래스(문자) 가중치 기본 설정
    weights = torch.ones(output_lang.n_chars, device=device)

    # 숫자 전체 가중치 증가
    for char, idx in output_lang.char2index.items():
        if char.isdigit():
            weights[idx] = 2.0

    # '0','1' 같이 혼동되는 숫자에 추가 가중치 (옵션)
    for c in ['0','1']:
        if c in output_lang.char2index:
            weights[output_lang.char2index[c]] = 2.5

    # 구분자 하이픈에 가중치
    if '-' in output_lang.char2index:
        weights[output_lang.char2index['-']] = 2.0
    if '/' in output_lang.char2index:
        weights[output_lang.char2index['/']] = 1.5

    # reduction='none'로 개별 토큰 손실 반환
    return nn.CrossEntropyLoss(weight=weights, reduction='none')

In [20]:
# 문자 단위 텐서 변환 함수
def char_tensor_from_sentence(lang, sentence):
    indexes = [lang.char2index[char] for char in sentence]
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# 모델 재생성 (vocabulary 크기가 바뀌었으므로)
hidden_size = 256
date_encoder = EncoderRNN(date_input_lang.n_chars, hidden_size).to(device)
date_decoder = BahdanauAttnDecoderRNN(hidden_size, date_output_lang.n_chars).to(device)

# 옵티마이저 재설정
date_encoder_optimizer = optim.Adam(date_encoder.parameters(), lr=0.0005)
date_decoder_optimizer = optim.Adam(date_decoder.parameters(), lr=0.0005)
date_optimizers = (date_encoder_optimizer, date_decoder_optimizer)
#criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()
criterion = create_weighted_criterion(date_output_lang)

# 학습 (문자 단위)
n_iters = 10000
#n_iters = 10
print_every = 1000

pbar = tqdm(range(1, n_iters + 1))
for iteration in pbar:
    pair = random.choice(raw_data)
    input_tensor = char_tensor_from_sentence(date_input_lang, pair[0]).transpose(0, 1)
    target_tensor = char_tensor_from_sentence(date_output_lang, pair[1]).transpose(0, 1)

    loss = train_bahdanau(input_tensor, target_tensor, date_encoder, date_decoder, date_optimizers, criterion)

    if iteration % print_every == 0:
        print(f"Iteration: {iteration} / {n_iters}, Loss: {loss:.4f}")

print("Character-level date format translation model training finished.")

  0%|          | 0/10000 [00:00<?, ?it/s]

 10%|█         | 1004/10000 [01:23<06:28, 23.17it/s]

Iteration: 1000 / 10000, Loss: 0.0840


 20%|██        | 2002/10000 [02:20<10:32, 12.64it/s]

Iteration: 2000 / 10000, Loss: 0.0058


...

Iteration: 8000 / 10000, Loss: 0.0002


 90%|█████████ | 9004/10000 [12:38<00:46, 21.35it/s]

Iteration: 9000 / 10000, Loss: 0.0001


100%|██████████| 10000/10000 [13:28<00:00, 12.36it/s]

Iteration: 10000 / 10000, Loss: 0.0000
Character-level date format translation model training finished.


In [21]:
def evaluate_char_date_translation(input_sentence, encoder, decoder):
    with torch.no_grad():
        input_tensor = char_tensor_from_sentence(date_input_lang, input_sentence).transpose(0, 1)
        encoder_outputs, encoder_hidden = encoder(input_tensor)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        decoded_chars = []
        decoder_attentions = []

        for di in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions.append(attn_weights.squeeze().cpu().numpy())

            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                break
            else:
                decoded_chars.append(date_output_lang.index2char[topi.item()])
            decoder_input = topi.squeeze().detach().view(1, -1)

        return ''.join(decoded_chars), decoder_attentions



In [22]:
# 테스트
test_examples = [
    "September 20, 2025",
    "18 Jun 2025",
    "2025/06/18",
    "06-18-2025"
]

print("=== Character-level Date Format Translation Results ===")
for test_input in test_examples:
    try:
        predicted_output, attentions = evaluate_char_date_translation(test_input, date_encoder, date_decoder)
        print(f"Input: {test_input}")
        print(f"Output: {predicted_output}")
        print("-" * 40)
    except Exception as e:
        print(f"Error with input '{test_input}': {e}")

=== Character-level Date Format Translation Results ===
Input: September 20, 2025
Output: 2025-09-20
----------------------------------------
Input: 18 Jun 2025
Output: 2025-06-18
----------------------------------------
Input: 2025/06/18
Output: 2025-06-18
----------------------------------------
Input: 06-18-2025
Output: 2025-06-18
----------------------------------------


In [23]:
def unit_test_date_translation(n_tests=100):
    """
    날짜 포맷 변환 모델의 단위 테스트 함수

    Args:
        n_tests: 테스트할 샘플 개수

    Returns:
        success_count, fail_count: 성공 및 실패 건수
    """
    success_count = 0
    fail_count = 0

    print(f"=== Unit Test for Date Translation Model ({n_tests} tests) ===")

    # 테스트 데이터 생성
    start_date = datetime(2020, 1, 1)

    for i in range(n_tests):
        try:
            # 테스트용 날짜 생성
            test_date = start_date + timedelta(days=random.randint(0, 2000))

            # INPUT_FORMATS 중 하나를 랜덤 선택
            input_format = random.choice(INPUT_FORMATS)

            # 입력 문자열 생성
            input_str = test_date.strftime(input_format)

            # 예상 출력 (정답)
            expected_output = test_date.strftime(OUTPUT_FORMATS)

            # 모델 예측
            predicted_output, attentions = evaluate_char_date_translation(input_str, date_encoder, date_decoder)

            # 결과 비교
            if predicted_output == expected_output:
                success_count += 1
                status = "PASS"
            else:
                fail_count += 1
                status = "FAIL"

            # 처음 10개와 실패한 케이스만 출력
            if i < 10 or predicted_output != expected_output:
                print(f"Test {i+1:3d}: {status}")
                print(f"  Input:     '{input_str}'")
                print(f"  Expected:  '{expected_output}'")
                print(f"  Predicted: '{predicted_output}'")
                print("-" * 50)

        except Exception as e:
            fail_count += 1
            print(f"Test {i+1:3d}: ERROR")
            print(f"  Input: '{input_str if 'input_str' in locals() else 'N/A'}'")
            print(f"  Error: {e}")
            print("-" * 50)

    # 결과 요약
    total_tests = success_count + fail_count
    success_rate = (success_count / total_tests * 100) if total_tests > 0 else 0

    print(f"\n=== Test Results Summary ===")
    print(f"Total Tests:   {total_tests}")
    print(f"Successful:    {success_count} ({success_rate:.1f}%)")
    print(f"Failed:        {fail_count} ({100-success_rate:.1f}%)")
    print(f"Success Rate:  {success_rate:.1f}%")

    return success_count, fail_count

# 단위 테스트 실행
success, fail = unit_test_date_translation(n_tests=50)

=== Unit Test for Date Translation Model (50 tests) ===
Test   1: FAIL
  Input:     '2021/01/11'
  Expected:  '2021-01-11'
  Predicted: '2021-11-11'
--------------------------------------------------
Test   2: PASS
  Input:     '04-07-2025'
  Expected:  '2025-04-07'
  Predicted: '2025-04-07'
--------------------------------------------------
Test   3: FAIL
  Input:     '31 Oct 2023'
  Expected:  '2023-10-31'
  Predicted: '2023-10-13'
--------------------------------------------------
Test   4: PASS
  Input:     '24 Sep 2021'
  Expected:  '2021-09-24'
  Predicted: '2021-09-24'
--------------------------------------------------
Test   5: FAIL
  Input:     '30 Jan 2020'
  Expected:  '2020-01-30'
  Predicted: '2020-06-03'
--------------------------------------------------
Test   6: FAIL
  Input:     '11-29-2021'
  Expected:  '2021-11-29'
  Predicted: '2021-11-21'
--------------------------------------------------
Test   7: PASS
  Input:     '2022/06/26'
  Expected:  '2022-06-26'
  Predicte